In [1]:
import mbuild as mb
import mdtraj as md
import numpy as np
from foyer.forcefield import Forcefield
from openff.toolkit.typing.engines.smirnoff.forcefield import ForceField

from openff.interchange.components.foyer import TopologyGraph
from openff.interchange.components.interchange import Interchange
from openff.interchange.components.mdtraj import _OFFBioTop

In [2]:
nanoparticle = mb.load("silica_nanoparticle.json")

In [3]:
oplsaa_silica = Forcefield("oplsaa_switchable.xml")

/Users/mwt/miniconda3/envs/interchange/lib/python3.7/site-packages/foyer/forcefield.py:658: UserWarning: Non-atomistic element type detected. Creating custom element for _EP
  "Creating custom element for {}".format(element)
/Users/mwt/miniconda3/envs/interchange/lib/python3.7/site-packages/foyer/forcefield.py:621: UserWarning: No force field version number found in force field XML file.
  "No force field version number found in force field XML file."
/Users/mwt/miniconda3/envs/interchange/lib/python3.7/site-packages/foyer/forcefield.py:633: UserWarning: No force field name found in force field XML file.
  "No force field name found in force field XML file."
/Users/mwt/miniconda3/envs/interchange/lib/python3.7/site-packages/foyer/forcefield.py:645: UserWarning: No combining rule found in force field XML file.
  "No combining rule found in force field XML file."
/Users/mwt/miniconda3/envs/interchange/lib/python3.7/site-packages/foyer/validator.py:165: ValidationWarning: You have empty s

In [4]:
import mdtraj as md
from openff.toolkit.typing.engines.smirnoff.forcefield import ForceField

from openff.interchange.components.foyer import TopologyGraph
from openff.interchange.components.interchange import Interchange
from openff.interchange.components.mdtraj import _OFFBioTop

In [5]:
topology = _OFFBioTop()
topology.mdtop = md.Topology.from_openmm(nanoparticle.to_parmed().topology)

In [6]:
nanoparticle_interchange = Interchange.from_foyer(
    topology=topology,
    force_field=oplsaa_silica,
)

In [7]:
nanoparticle_interchange.positions = nanoparticle.xyz

In [8]:
from openff.toolkit.topology import Molecule

In [9]:
molecule = Molecule.from_iupac("aspirin")
molecule.generate_conformers(n_conformers=1)

In [10]:
sage = ForceField("openff_unconstrained-2.0.0.offxml")

In [11]:
aspirin_interchange = Interchange.from_smirnoff(sage, molecule.to_topology())

In [12]:
from openff.units import unit
from openmm import unit as openmm_unit

In [13]:
aspirin_interchange.positions = molecule.conformers[0] + 5 * openmm_unit.nanometer

In [14]:
combined = aspirin_interchange + nanoparticle_interchange

/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:582: UserWarning: Interchange object combination is experimental and likely to produce strange results. Any workflow using this method is not guaranteed to be suitable for production. Use with extreme caution and thoroughly validate results!
  "Interchange object combination is experimental and likely to produce "
/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:614: UserWarning: 'other' Interchange object has handler with name RBTorsions not found in 'self,' but it has now been added.
  f"'other' Interchange object has handler with name {handler_name} not "
/Users/mwt/software/openff-system/openff/interchange/components/interchange.py:614: UserWarning: 'other' Interchange object has handler with name RBImpropers not found in 'self,' but it has now been added.
  f"'other' Interchange object has handler with name {handler_name} not "


In [15]:
from openff.interchange.drivers.gromacs import get_gromacs_energies

In [16]:
get_gromacs_energies(combined)

EnergyReport(energies={'Bond': <Quantity(24921.6367, 'kilojoule / mole')>, 'Angle': <Quantity(35966.5195, 'kilojoule / mole')>, 'Torsion': <Quantity(43.0900764, 'kilojoule / mole')>, 'vdW': <Quantity(6417.12457, 'kilojoule / mole')>, 'Electrostatics': <Quantity(-272787.339, 'kilojoule / mole')>})